1. Implement two classification algorithms for detecting faces and classifying digits:
(a) Perceptron
(b) Two-layer Neural Network (input layer, one hidden layer, output)
2. Design the features for each of the two problems, and write a program for extracting the features from each image.
You can also use the raw pixels directly as features (and avoid the process of designing features altogether) if that
results in a better classification accuracy.
3. Train the two algorithms on the part of the data set that is reserved for training. First, use only 10% of the data points
that are reserved for training, then 20%, 30%, 40%, 50%, 60%, 70%, 80%, 90%, and finally 100%. All the results
should a function of the number of data points used for training.
4. Compare the performances of the two algorithms using the part of the data set that is reserved for testing, and report:
• The time needed for training as a function of the number of data points used for training.
• The prediction error (and standard deviation) as a function of the number of data points used for training.
5. Write a report describing the implemented algorithms and discussing the results and the learned lessons.

In [1]:
# if f(x) >= 0 then y = face
# else y = not face

#f(x) = w1*phi(x) + w2*phi2(x) + ..
# phi(x) = number of black pixels (example)

# intialize weights randomonly at first

# training condition is while less than 5 minutes or
# the weights aren't changing from image to image

#  update weights in training
 # if f(x) >=0 and y =face
 # or  f(x) <= 0 and y = not face
 # then do nothing

 # else if f(x) >= 0 and y = not face
 # then w1 = w1 -phi1 (x) 
 #      w2 = w2 - phi2(x)

# else if f(x) < 0 and y = face
 # then w1 = w1 + phi1 (x) 
 #      w2 = w2 + phi2(x)


#f = w1 * phi1(x) + w2 * phi2(x)



In [2]:
# Determine features - will need to obtain from image
# First step is get data from facedatatrain to use in this notebook
# Each face image is 70x60 (row x column) and each face begins and ends with an empty line (not including the 70x60).
# Then make a feature matrix of size 70*60, where each pixel is a feature

In [2]:
! pip install numpy

You should consider upgrading via the 'C:\Users\nafri\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [3]:
import sys
sys.path.insert(1, 'C:/Users/nafri/College/Senior Year/Spring Semester/Intro to AI/AI-FinalProject/Final Project/Berkeley_Example')
sys.path.insert(1, 'C:/Users/jzlyn/OneDrive/Desktop/CS440/AI-FinalProject/Final Project/Berkeley_Example')

In [4]:
from Berkeley_Example import samples
from Berkeley_Example.perceptron import PerceptronClassifier
import numpy as np
import math

In [3]:
def sigmoid(x):
    try:
        sig =1/(1 + math.exp(-x))
    except:
        sig = 0   
    return sig

In [60]:
def update_weights(image,weights,over_estimate):
    for i in range(len(weights)):
        for j in range(len(weights[0])):
            if over_estimate:
                weights[i][j] -= image[i][j]
            else:
                weights[i][j] += image[i][j]

In [61]:
def face_train(n,percent):
    face_train_images = samples.loadDataFile('data/facedata/facedatatrain', n, 60, 70)
    face_train_labels = samples.loadLabelsFile('data/facedata/facedatatrainlabels', n)
    
    size = n * percent

    weights = np.random.uniform(-1, 1, size=(70, 60))
    bias = np.random.uniform(-1, 1)
    f = np.zeros(int(size)).astype(float)

    image_lst = []

    rand_samples = np.random.choice(n, int(size), replace=False)
    
    for i in rand_samples:
        matrix = face_train_images[i].getPixels()
        for r in range(len(matrix)):
            for c in range(len(matrix[0])):
                matrix[r][c] = samples.IntegerConversionFunction(matrix[r][c])
        image_lst.append(matrix)
            

    for i in range(len(image_lst)):
        f[i] = bias
        image = image_lst[i]
        for r in range(len(image)):
            for c in range(len(image[0])):
                f[i] += weights[r][c] * image[r][c]

        if f[i] >= 0 and face_train_labels[rand_samples[i]] == 0:
            update_weights(image,weights,True)
            bias -= 1
        elif f[i] < 0 and face_train_labels[rand_samples[i]] == 1:
            update_weights(image,weights,False)
            bias += 1

    return (weights, bias)  

In [62]:
def face_validate(n,percent,weights,bias):
    face_valid_images = samples.loadDataFile('data/facedata/facedatavalidation', n, 60, 70)
    face_valid_labels = samples.loadLabelsFile('data/facedata/facedatavalidationlabels', n)

    size = n * percent
 
    f = np.zeros(int(size)).astype(float)

    image_lst = []

    rand_samples = np.random.choice(n, int(size), replace=False)
    
    for i in rand_samples:
        matrix = face_valid_images[i].getPixels()
        for r in range(len(matrix)):
            for c in range(len(matrix[0])):
                matrix[r][c] = samples.IntegerConversionFunction(matrix[r][c])
        image_lst.append(matrix)

    for i in range(len(image_lst)):
        f[i] = bias
        image = image_lst[i]
        for r in range(len(image)):
            for c in range(len(image[0])):
                f[i] += weights[r][c] * image[r][c]

    num_correct = 0
    for i in range(len(f)):
        if (f[i] >= 0 and face_valid_labels[rand_samples[i]] == 1) or (f[i] < 0 and face_valid_labels[rand_samples[i]] == 0):
            num_correct += 1

    result = num_correct / size

    return result

In [63]:
TRAIN_SIZE = 451
VALID_SIZE = 301
epochs = 5
percentage = np.arange(0.1,1.1,.1)

for percent in percentage:
    accuracy = []
    for i in range(epochs):
        weights, bias = face_train(TRAIN_SIZE,percent) # return weights and bias used for validation
        result = face_validate(VALID_SIZE,percent,weights,bias)
        accuracy.append(result)

    # Optional - Save to file later    
    print(f'Percent of Training Data Used = {"{:.2%}".format(percent)}')
    print(accuracy)
    print(f'Mean: {np.mean(accuracy)}')
    print(f'Std: {np.std(accuracy)}')
    print()

Percent of Training Data Used = 10.00%
[0.8637873754152824, 0.7641196013289037, 0.5315614617940199, 0.5647840531561461, 0.5980066445182723]
Mean: 0.6644518272425249
Std: 0.12780986087489268

Percent of Training Data Used = 20.00%
[0.6810631229235881, 0.7973421926910299, 0.6810631229235881, 0.6976744186046512, 0.6976744186046512]
Mean: 0.7109634551495017
Std: 0.04382360783479375

Percent of Training Data Used = 30.00%
[0.7973421926910298, 0.8416389811738648, 0.830564784053156, 0.6423034330011074, 0.6866002214839423]
Mean: 0.7596899224806201
Std: 0.08034709699763921

Percent of Training Data Used = 40.00%
[0.7807308970099668, 0.7308970099667774, 0.8056478405315615, 0.8637873754152824, 0.7641196013289037]
Mean: 0.7890365448504982
Std: 0.04457278360464697

Percent of Training Data Used = 50.00%
[0.7973421926910299, 0.7308970099667774, 0.8305647840531561, 0.717607973421927, 0.7906976744186046]
Mean: 0.7734219269102991
Std: 0.04256642492504176

Percent of Training Data Used = 60.00%
[0.81395

In [70]:
def update_digit_weights(image,weights,bias,prediction,actual):
    bias[prediction] -= 1
    bias[actual] += 1
    for i in range(len(weights)):
        for j in range(len(weights[0])):
            weights[prediction][i][j] -= image[i][j]
            weights[actual][i][j] += image[i][j]

In [84]:
def digit_train(n,percent):
    digit_train_images = samples.loadDataFile('data/digitdata/trainingimages', n, 28, 28)
    digit_train_labels = samples.loadLabelsFile('data/digitdata/traininglabels', n)

    

    for i in range(5):
    
    size = n * percent
    DIGITS = 10
    weights = np.random.randint(-400, 400, size=(DIGITS, 28, 28))
    #weights = np.random.randint(-5, 5, size=(DIGITS, 28, 28))
    #print(weights)
    bias = np.random.randint(-200, 200, size=DIGITS)
    #bias = np.random.randint(-5, 5, size=DIGITS)
    #print(bias)
    #f = np.zeros(int(size)).astype(float)
    prediction = 0
    #print(f)
    accuracy = []
    image_lst = []

    rand_samples = np.random.choice(n, size, replace=False)
    #rand_samples = [0,1,2,3,4]
    
    for i in rand_samples:
        matrix = digit_train_images[i].getPixels()
        for r in range(len(matrix)):
            for c in range(len(matrix[0])):
                matrix[r][c] = samples.IntegerConversionFunction(matrix[r][c])
        #print(matrix)
        image_lst.append(matrix)

    #print(rand_samples)
            

    for i in range(len(image_lst)):
        max_predict = np.NINF
        arg_max = 0
        prediction = 0
        for d in range(0,10):
            #f[i] = bias[d]
            #prediction = bias[d]
            image = image_lst[i]
            for r in range(len(image)):
                for c in range(len(image[0])):
                    prediction += weights[d][r][c] * image[r][c]
            #print(f'f digit {d} image {i}: {f[d][i]}')
            prediction += bias[d]
            if prediction > max_predict:
                max_predict = prediction
                arg_max = d
        
        #print(f'Predicted Digit: {arg_max} Actual Digit: {digit_train_labels[rand_samples[i]]}')
        #print()

        if arg_max == digit_train_labels[rand_samples[i]]:
            accuracy.append(1)
        else:
            accuracy.append(0)

        if arg_max != digit_train_labels[rand_samples[i]]:
            #print(f'Before update: Bias Predict = {bias[arg_max]} Bias Actual = {bias[digit_train_labels[rand_samples[i]]]}')
            update_digit_weights(image,weights,bias,arg_max,digit_train_labels[rand_samples[i]])
            #print(f'After update: Bias Predict = {bias[arg_max]} Bias Actual = {bias[digit_train_labels[rand_samples[i]]]}')

    print(np.mean(accuracy))

        

    return (weights, bias)  

In [85]:
digit_train(5000,1)

0.1838


(array([[[ 305, -123,  342, ..., -298,   28,  118],
         [-132,  -62, -280, ...,  -40,  272, -149],
         [-293,  339,   20, ...,  -79,  111, -359],
         ...,
         [ 275, -133,  -37, ...,  384,  389,  -60],
         [-398,   40, -329, ...,  124, -240, -201],
         [  38,   73, -130, ...,  -31,  224,  395]],
 
        [[ 272, -390,  244, ...,  304,  309,   92],
         [ 311,   57,  187, ...,  -67,    6,  -53],
         [ 284, -377,  260, ...,  235,   86,  330],
         ...,
         [ 262, -119,  -72, ..., -251, -247,  128],
         [-390, -252, -202, ...,   66, -316,  -40],
         [-327, -242, -392, ...,  362,  164, -259]],
 
        [[-281, -118,    7, ...,   23, -116,   26],
         [ -74,  148, -328, ..., -206, -302,  -12],
         [ 107, -207,  334, ..., -376, -131,  347],
         ...,
         [-390, -151, -399, ...,  248,   26,  180],
         [ 395,  149, -172, ...,  -25, -292,  107],
         [-297,  310, -315, ...,  -72, -197,   76]],
 
        ...,


In [78]:
def digit_validate(n,percent,weights,bias):
    digit_valid_images = samples.loadDataFile('data/digitdata/validationimages', n, 28, 28)
    digit_valid_labels = samples.loadLabelsFile('data/digitdata/validationlabels', n)

    size = n * percent
    #print(size)
    DIGITS = 10
 
    #f = np.zeros(int(size)).astype(float)
    #print(f)
    prediction = 0

    image_lst = []

    rand_samples = np.random.choice(n, size, replace=False)
    
    for i in rand_samples:
        matrix = digit_valid_images[i].getPixels()
        for r in range(len(matrix)):
            for c in range(len(matrix[0])):
                matrix[r][c] = samples.IntegerConversionFunction(matrix[r][c])
        image_lst.append(matrix)
        
    accuracy = []
    #prediction = []
    for i in range(len(image_lst)):
        max_predict = np.NINF
        arg_max = 0 
        prediction = 0
        for d in range(DIGITS):
            #f[i] = bias[d]
            #prediction = bias[d]
            image = image_lst[i]
            for r in range(len(image)):
                for c in range(len(image[0])):
                    #f[i] += weights[d][r][c] * image[r][c]
                    prediction += weights[d][r][c] * image[r][c]
            #print(f'f digit {d} image {i}: {f[i]}')
            prediction += bias[d]
            if prediction > max_predict:
                max_predict = prediction
                arg_max = d

        if arg_max == digit_valid_labels[rand_samples[i]]:
            accuracy.append(1)
        else:
            accuracy.append(0)
            

        #print()
            
        #prediction.append(arg_max)

    '''
    num_correct = 0
    for i in range(len(prediction)):
        #print(f'Predicted Digit: {prediction[i]} Actual Digit: {digit_valid_labels[rand_samples[i]]}')
        if prediction[i] == digit_valid_labels[rand_samples[i]]:
            num_correct += 1

    result = num_correct / size
    '''

    return np.mean(accuracy)

In [79]:
TRAIN_SIZE = 5000
VALID_SIZE = 1000
epochs = 5
#percentage = [1] #np.arange(0.1,1.1,.1)
percent = 1
#for percent in percentage:
accuracy = []
for i in range(epochs):
    weights, bias = digit_train(TRAIN_SIZE,percent) # return weights and bias used for validation
    result = digit_validate(VALID_SIZE,percent,weights,bias)
    accuracy.append(result)

    # Optional - Save to file later    
    print(f'Percent of Training Data Used = {"{:.2%}".format(percent)}')
    print(accuracy)
    print(f'Mean: {np.mean(accuracy)}')
    print(f'Std: {np.std(accuracy)}')
    print()

Percent of Training Data Used = 100.00%
[0.447]
Mean: 0.447
Std: 0.0



KeyboardInterrupt: 

In [11]:
import numpy as np
weights = np.random.rand(1, 4200)
phitures = 0